In [ ]:
#NOTEBOOK 1: data cleaning, feature selection e fitting di modelli di classificazione, con analisi delle performance e degli errori commessi.
#Le prime 2 celle sono dedicate a settare la rete Word2Vec in italiano che serve a estrarre features semantiche
#dalle descrizioni delle fatture in modo da distinguere meglio le classi

!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!sudo pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3910, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 3910 (delta 19), reused 24 (delta 2), pack-reused 3854
Receiving objects: 100% (3910/3910), 8.33 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (2436/2436), done.
/content/fastText
Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3124345 sha256=294c60a843c7e8383d9ca0a6c9f9aef0b4630d165e336d342782a7

In [ ]:
import fasttext
import fasttext.util
fasttext.util.download_model('it', if_exists='ignore')
ft = fasttext.load_model('cc.it.300.bin')
print(ft.get_dimension())


300


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#lettura delle descrizioni dal dataset

from scipy import spatial
import pandas as pd
!pip install --upgrade openpyxl

descr = pd.read_excel('/content/drive/MyDrive/df_final.xlsx')
descr = descr['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_Descrizione']

print(list(descr))

['A513-77199580-8000500131329-NUTELLA G3000X2 SCC/PRT F/SER', 'CONTRIBUTO CONAI-A513-77199580-8000500131329-NUTELLA G3000X2 SCC/PRT F/SER', 'ANTICIPO PER PAGAMENTO RICEVUTO', 'Rif. Ordine n.r. 109763889', 'Rif. Ord.Cli. ORD109763889 del 13/04/2021', 'POSTA ELETTRONICA CERTIFICATA', 'updownbarsrl@pec.it', 'Durata Servizio dal 3/5/2021 al 3/5/2022', 'Rif. Ord.Cli. ORD109763889 del 13/04/2021', 'Detratto anticipo', 'AB3 MISCELA CREMA BAR CONFEZIONE DA Kg.3', '/D Lotto V38I', 'N ZUCCHERO SEMOLATO ASTUCCI da Kg.1', '/D Lotto 1025F0892R', 'OZD DOLCIFICANTE IN DISPENSER DA 240 BST', '/D Lotto 5023F0798G', '002 IL CREMINO CONF. Kg.1', '/D Lotto LE062', "TOTN OMAGGIO TAZZE PUBBLICITARIE ALL.DICH.CONFORMITA'", "PIATN OMAGGIO PIATTINI PER TAZZE PUBBL.RIE ALL.DICH.CONFORMITA'", 'AMBROSIO FRUTTA MISTA INTERA CT 5 KG', 'IRCA MIX DOLCE FORNO CF 25 KG COD. 1070116', 'GAMMA SOTTOGELO GRANDI PER CASSATINE CF 1000 PZ', 'MANDORLE SPAGNA NATURALI ***1KG***', 'VANDEMOORTELE ZUCCHERO FONDENTE CF 15 KG', 'Spe

In [ ]:
#adesso, per ogni descrizione, si devono assegnare 4 feature di presumibile appartenenza alle 4 classi critiche. Per
#ciascuna classe vengono selezionate 2 parole semanticamente simili ai prodotti classificati in tale classe e viene valutata,
#grazie all'embedding fornito da Word2Vec, la similarità semantica delle varie parole presenti in una descrizione con queste
#2 parole di riferimento, per tutte e 4 le classi. Per ogni descrizione viene poi tenuto solo lo score più alto ottenuto dalle sue parole per individuare
#quella chiave

similarity_materie_prime = []
similarity_materie_consumo = []
similarity_merci = []
similarity_merci_prodotti = []

vector_formaggio = ft.get_word_vector('formaggio')
vector_uova = ft.get_word_vector('uova')
vector_busta = ft.get_word_vector('busta')
vector_posate = ft.get_word_vector('posate')
vector_kinder = ft.get_word_vector('kinder')
vector_caramella = ft.get_word_vector('caramella')
vector_bibita = ft.get_word_vector('bibita')
vector_vodka = ft.get_word_vector('vodka')

for s in list(descr):
  s = s.lower()
  s_words = s.split(" ")

  words_similarities_materie_prime = []
  words_similarities_materie_consumo = []
  words_similarities_merci = []
  words_similarities_merci_prodotti = []

  for i in s_words:
    vector_i = ft.get_word_vector(i)
    score_materie_prime = ((1 - spatial.distance.cosine(vector_i, vector_formaggio)) + (1 - spatial.distance.cosine(vector_i, vector_uova)))/2
    score_materie_consumo = ((1 - spatial.distance.cosine(vector_i, vector_busta)) + (1 - spatial.distance.cosine(vector_i, vector_posate)))/2
    score_merci = ((1 - spatial.distance.cosine(vector_i, vector_kinder)) + (1 - spatial.distance.cosine(vector_i, vector_caramella)))/2
    score_merci_prodotti = ((1 - spatial.distance.cosine(vector_i, vector_bibita)) + (1 - spatial.distance.cosine(vector_i, vector_vodka)))/2

    words_similarities_materie_prime.append(score_materie_prime)
    words_similarities_materie_consumo.append(score_materie_consumo)
    words_similarities_merci.append(score_merci)
    words_similarities_merci_prodotti.append(score_merci_prodotti)

  words_similarities_materie_prime = [x for x in words_similarities_materie_prime if pd.notnull(x)]
  words_similarities_materie_consumo = [x for x in words_similarities_materie_consumo if pd.notnull(x)]
  words_similarities_merci = [x for x in words_similarities_merci if pd.notnull(x)]
  words_similarities_merci_prodotti = [x for x in words_similarities_merci_prodotti if pd.notnull(x)]

  best_materie_prime = max(words_similarities_materie_prime)
  best_materie_consumo = max(words_similarities_materie_consumo)
  best_merci = max(words_similarities_merci)
  best_merci_prodotti = max(words_similarities_merci_prodotti)

  similarity_materie_prime.append(best_materie_prime)
  similarity_materie_consumo.append(best_materie_consumo)
  similarity_merci.append(best_merci)
  similarity_merci_prodotti.append(best_merci_prodotti)



/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
#lettura del file excel in pandas

import pandas as pd
!pip install --upgrade openpyxl

data = pd.read_excel('/content/drive/MyDrive/df_final.xlsx')
data = data.drop('Unnamed: 0', axis=1)
print(data.head(5))
print(data.columns)

In [ ]:
#selezione delle colonne di interesse

data = data[['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale',
            'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA',
            'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice',
            'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data',
            'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT',
            'Conto']]

print(data.shape)
print(data.head(5))

In [ ]:
#aggiunta delle colonne con le informazioni estratte dalla descrizione precedentemente create

data = pd.concat(objs=[data, descr.rename('Descrizione'), pd.Series(similarity_materie_prime).rename('Similarity_Materie_Prime'), pd.Series(similarity_materie_consumo).rename('Similarity_Materie_Consumo'), pd.Series(similarity_merci).rename('Similarity_Merci'), pd.Series(similarity_merci_prodotti).rename('Similarity_Merci_Prodotti')], axis=1)
print(data.shape)

In [ ]:
#controllo del tipo delle colonne

for column in data.columns:
  print(column)
  print(type(data[column][0]))

FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario
<class 'numpy.float64'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale
<class 'numpy.float64'>
FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA
<class 'numpy.float64'>
FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice
<class 'numpy.int64'>
FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT
<class 'str'>
Conto
<class 'str'>
Descrizione
<class 'str'>
Similarity_Materie_Prime
<class 'numpy.float64'>
Similarity_Materie_Consumo
<class 'numpy.float64'>
Similarity_Merci
<class

In [5]:
#conversione del tipo di dato in alcune colonne

from datetime import datetime

data['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA'] = data['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA'].astype(int)
data['FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice'] = data['FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice'].astype(str)


In [6]:
#conversione della data in 3 colonne giorno-mese-anno per renderle feature adatte a fare machine learning

year = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).year)
month = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).month)
day = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).day)

frame = {'year': year, 'month': month, 'day': day}
data2 = pd.DataFrame(frame)
  
data = pd.concat(objs=[data, data2], axis=1)
print(data.columns)

Index(['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale',
       'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA',
       'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice',
       'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data',
       'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT', 'Conto',
       'year', 'month', 'day'],
      dtype='object')


In [7]:
#eliminazione della colonna con la data in formato timestamp

data.drop('FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data', axis=1, inplace=True)
print(data.columns)

Index(['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale',
       'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA',
       'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice',
       'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT', 'Conto',
       'year', 'month', 'day'],
      dtype='object')


In [8]:
#rinominazione delle colonne

data = data.rename(columns={'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero': 'Numero',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first': 'CodiceTipo',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario': 'PrezzoUnitario',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale': 'PrezzoTotale',
            'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first': 'ModalitaPagamento',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA': 'AliquotaIVA',
            'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice': 'CodiceCedente',
            'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT': 'NumeroDDT',
            'Conto': 'Conto', 'year': 'Year', 'month': 'Month', 'day': 'Day'})

print(data.columns)

Index(['Numero', 'CodiceTipo', 'PrezzoUnitario', 'PrezzoTotale',
       'ModalitaPagamento', 'AliquotaIVA', 'CodiceCedente', 'NumeroDDT',
       'Conto', 'Year', 'Month', 'Day'],
      dtype='object')


In [9]:
#eliminazione, in tutte le colonne tranne 'Conto', dei nan che compaiono come float a favore di stringhe vuote (perche tutte le colonne di feature con dei nan sono in formato stringa)
Conti = data.Conto
data = data.fillna('')
data.Conto = Conti
print(data.isnull().sum().sum())


4061


In [ ]:
#analisi della distribuzione della variabile target

print(data.groupby('Conto').count())
print(data.shape)

In [11]:
#rimozione delle unità statistiche con target troppo rari (threshold = 5, incluso). La molteplicità dei target scende da 24 a 13.

mask = (data['Conto'] != '18/40/501')&(data['Conto'] != '66/25/505')&(data['Conto'] != '66/25/508')&(data['Conto'] != '66/30/060')&(data['Conto'] != '68/05/005')&(data['Conto'] != '68/05/133')&(data['Conto'] != '68/05/290')&(data['Conto'] != '68/05/320')&(data['Conto'] != '68/05/385')&(data['Conto'] != '68/05/407')&(data['Conto'] != '88/20/035')
data = data[mask]
print(data.shape)

(5855, 12)


In [12]:
#semplificazione N1: tutte le features categoriche vengono gestite col semplice One Hot Encoding, anche se questo causa
#un grande aumento del numero di features. Altri encoders potrebbero performare meglio (es: hashing trick).
#semplificazione N2: non si considera la struttura gerarchica dei conti ma le labels vengono predette come fossero 13 classi indipendenti
#semplificazione N3: si utilizza un solo modello predittivo anzichè dividere inizialmente il dataset e usare un modello su 
#ogni porzione. Ad esempio, potrebbe essere interessante dividere il dataset a seconda dell'aliquota IVA dato che questa
#discerne quasi perfettamente le classi 66/05/006 e 66/20/005, sulle quali continuano a essere commessi errori
#semplificazione N4: si valutano le performance con una semplice divisione train-test senza ricorrere alla CV
#semplificazione N5: non si risistemano le stringe del CodiceTipo che potrebbero essere formattate meglio dando vita a meno
#classi binarie

#N1
ModalitaPagamento_encoded = pd.get_dummies(data.ModalitaPagamento, prefix='ModalitaPagamento')
print(ModalitaPagamento_encoded.shape)
CodiceTipo_encoded = pd.get_dummies(data.CodiceTipo, prefix='CodiceTipo')
print(CodiceTipo_encoded.shape)
NumeroDDT_encoded = pd.get_dummies(data.NumeroDDT, prefix='NumeroDDT')
print(NumeroDDT_encoded.shape)
CodiceCedente_encoded = pd.get_dummies(data.CodiceCedente, prefix='CodiceCedente')
print(CodiceCedente_encoded.shape)

data.drop(['CodiceTipo', 'ModalitaPagamento', 'CodiceCedente', 'NumeroDDT'], axis=1, inplace=True)
data = pd.concat(objs=[data, CodiceTipo_encoded, ModalitaPagamento_encoded, CodiceCedente_encoded, NumeroDDT_encoded], axis=1)

print(data.shape)
print(data.columns)

#si arriva ad avere 204 features, a causa dei One Hot Encoding

(5855, 8)
(5855, 22)
(5855, 109)
(5855, 55)
(5855, 202)
Index(['Numero', 'PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA', 'Conto',
       'Year', 'Month', 'Day', 'CodiceTipo_', 'CodiceTipo_ARTICOLO',
       ...
       'NumeroDDT_PKL.21.1321', 'NumeroDDT_PKL.21.1457',
       'NumeroDDT_PKL.21.161', 'NumeroDDT_PKL.21.195', 'NumeroDDT_PKL.21.471',
       'NumeroDDT_PKL.21.520', 'NumeroDDT_PKL.21.55', 'NumeroDDT_PKL.21.597',
       'NumeroDDT_PKL.21.914', 'NumeroDDT_PKL.21.97'],
      dtype='object', length=202)


In [13]:
#riallocazione della colonna di target come ultima

temp = data.Conto
data.drop('Conto', axis=1, inplace=True)
data = pd.concat(objs=[data, temp], axis=1)
print(data.columns)

Index(['Numero', 'PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA', 'Year',
       'Month', 'Day', 'CodiceTipo_', 'CodiceTipo_ARTICOLO',
       'CodiceTipo_Articolo',
       ...
       'NumeroDDT_PKL.21.1457', 'NumeroDDT_PKL.21.161', 'NumeroDDT_PKL.21.195',
       'NumeroDDT_PKL.21.471', 'NumeroDDT_PKL.21.520', 'NumeroDDT_PKL.21.55',
       'NumeroDDT_PKL.21.597', 'NumeroDDT_PKL.21.914', 'NumeroDDT_PKL.21.97',
       'Conto'],
      dtype='object', length=202)


In [ ]:
#in vista del semi supervised learning, separazione dei dati supervisionati e non supervisionati

from sklearn.semi_supervised import SelfTrainingClassifier

data_labeled = data[data['Conto'].notna()]
data_unlabeled = data[data['Conto'].isnull()]

data_labeled.reset_index(inplace=True)
data_labeled = data_labeled.drop('index', axis=1)
data_unlabeled.reset_index(inplace=True)
data_unlabeled = data_unlabeled.drop('index', axis=1)

print(data_labeled)
print(data_unlabeled)

#si usa solo la porzione di dati labeled in un primo train-test di alcuni classificatori

In [ ]:
#primo classificatore provato: Naive Bayes, con diverse assunzioni sulla distribuzione delle features condizionata al target. 

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

features = data_labeled.drop(['Conto'], axis=1)
targets = data_labeled.Conto

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
gnb = CategoricalNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(1-((y_test != y_pred).sum()/ X_test.shape[0]))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#accuracy performance: GaussianNB: 0.79, BernoulliNB: 0.83

In [ ]:
#secondo classificatore provato: Decision Tree

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel


features = data_labeled.drop(['Conto', 'Descrizione', 'Numero'], axis=1)
targets = data_labeled.Conto
print(features.shape)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
clf = tree.DecisionTreeClassifier(max_depth=28)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(1-((y_test != y_pred).sum()/ X_test.shape[0]))
#print(clf.feature_importances_)

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
#print(f1_score(y_test, y_pred, average='micro')), è inutile tanto è uguale all'accuracy
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#riduzione delle features secondo il criterio di importanza basato sul Gini index. Con la soglia scelta di default
#si scende di solito a 26 features

model = SelectFromModel(clf, prefit=True)
features_new = model.transform(features)
print(features_new.shape)

X_train_new, X_test_new, y_train, y_test = train_test_split(features_new, targets, test_size=0.3, shuffle=False)
clf2 = tree.DecisionTreeClassifier(max_depth=15)
y_pred2 = clf2.fit(X_train_new, y_train).predict(X_test_new)
print(1-((y_test != y_pred2).sum()/ X_test_new.shape[0]))
#print(clf2.feature_importances_)

#le performance scendono leggermente da circa 0.93 a 0.91

(1794, 204)
0.9239332096474954
[0.93548387 0.93714286 0.87804878 0.9        0.97435897 0.66666667
 0.72727273 1.         0.94117647 1.         1.        ]
0.905468213407728
0.9239032324174937
(1794, 25)
0.9202226345083488


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
!pip install graphviz

In [ ]:
import graphviz
import pydotplus
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=features.columns,  
                                class_names=targets,
                                filled=True)

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_png('/content/original_tree.png')
pydot_graph.set_size('"5,5!"')
pydot_graph.write_png('/content/resized_tree.png')
pydot_graph
# Draw graph
#graph = graphviz.Source(dot_data, format="png") 
#graph


In [ ]:
#terzo classificatore provato: Random Forest

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


features = data_labeled.drop(['Conto', 'Descrizione', 'Numero'], axis=1)
targets = data_labeled.Conto
descr = data_labeled.Descrizione
num = data_labeled.Numero
print(features.shape)

"""
scaler = StandardScaler()
features.PrezzoUnitario = scaler.fit_transform(np.array(features.PrezzoUnitario).reshape(-1, 1))
features.PrezzoTotale = scaler.fit_transform(np.array(features.PrezzoTotale).reshape(-1, 1))
features.AliquotaIVA = scaler.fit_transform(np.array(features.AliquotaIVA).reshape(-1, 1))
features.Year = scaler.fit_transform(np.array(features.Year).reshape(-1, 1))
features.Month = scaler.fit_transform(np.array(features.Month).reshape(-1, 1))
features.Day = scaler.fit_transform(np.array(features.Day).reshape(-1, 1))
features.Similarity_Materie_Prime = scaler.fit_transform(np.array(features.Similarity_Materie_Prime).reshape(-1, 1))
features.Similarity_Materie_Consumo = scaler.fit_transform(np.array(features.Similarity_Materie_Consumo).reshape(-1, 1))
features.Similarity_Merci = scaler.fit_transform(np.array(features.Similarity_Merci).reshape(-1, 1))
features.Similarity_Merci_Prodotti = scaler.fit_transform(np.array(features.Similarity_Merci_Prodotti).reshape(-1, 1))

X_train = features.loc[540:,]
X_test = features.loc[:539,]
y_train = targets.loc[540:,]
y_test = targets.loc[:539,]
"""

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
clf = RandomForestClassifier(n_estimators=50, max_depth=16, bootstrap = False)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))
#print(clf.feature_importances_)

"""

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
#print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#Con la soglia scelta di default si scende di solito a 36 features

model = SelectFromModel(clf, prefit=True)
features_new = pd.DataFrame(model.transform(features))
print(features_new.shape)
print(features_new.head(5))

X_train_new, X_test_new, y_train, y_test = train_test_split(features_new, targets, test_size=0.3, shuffle=False)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=15, bootstrap = False)
y_pred2 = clf2.fit(X_train_new, y_train).predict(X_test_new)
print(accuracy_score(y_test, y_pred2))

print(f1_score(y_test, y_pred2, average=None))
#print(f1_score(y_test, y_pred2, average='micro'))
print(f1_score(y_test, y_pred2, average='macro'))
print(f1_score(y_test, y_pred2, average='weighted'))

#le performance, in quanto ad accuracy, sono paragonabili tra il modello con 204 features e quello ridotto e intorno 
#a 0.94/0.95. E' stata testata anche una pipeline che prevede prima una standardizzazione dei dati ma senza riportare
#significative variazioni nella performance. Anche F-score assume più o meno gli stessi valori (nella versione weighted che tiene)
#conto della dimensione delle classi). Invece la versione macro è potenzialmente anche più alta dell'accuracy perchè, come si 
#vede dalla confusion matrix, gli errori sono di solito sulle classe più grandi; a volte però si abbassa quando non viene appresa la classe
#66/25/509, che ha poche istanze, e quindi il suo F score è 0.
#Per sopperire alla momentanea scarsa precisione nel determinare l'accuracy a causa della mancata CV è stato anche usato come
#test set la parte iniziale del dataset e viceversa per il training set rilevando le stesse performance che quindi si 
#possono considerare veritiere
"""

(1816, 207)


ValueError: ignored

In [ ]:
clf = RandomForestClassifier(n_estimators=50, max_depth=16, bootstrap = False)
clf.fit(features, targets)

RandomForestClassifier(bootstrap=False, max_depth=16, n_estimators=50)

In [ ]:
import pickle
filename = '/content/finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9944954128440368


In [ ]:
#feature selection basata sul random forest, basata sulle features scelte inizialmente

mask = clf.feature_importances_ > 0.03
print(features.columns[mask])

#Risultano importanti il prezzo unitario, il prezzo totale, l'IVA, il giorno (???), la modalità di pagamento e il codice cedente
#Risultano poco importanti il prezzo totale, il mese e l'unità di misura
#Risultano non importanti il codice cessionario, il tipo di documento, la condizione di pagamento e l'anno. In effetti queste features assumono sempre (o quasi)
#lo stesso valore in questo dataset

#Ho levato sin dall'inizio le features inutili (anche l'unità di misura) tenendo solo l'anno che può rivelarsi utile per un dataset 
#pluriennale. Le performance non ne risentono


Index(['PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA',
       'Similarity_Materie_Prime', 'Similarity_Materie_Consumo',
       'Similarity_Merci', 'Similarity_Merci_Prodotti', 'Day',
       'CodiceTipo_AswArtFor', 'ModalitaPagamento_MP01',
       'ModalitaPagamento_MP19', 'CodiceCedente_10483110010',
       'CodiceCedente_183410653', 'CodiceCedente_4352551214',
       'CodiceCedente_777280157'],
      dtype='object')


In [ ]:
#quarto classificatore provato: AdaBoost

from sklearn.ensemble import AdaBoostClassifier

features = data_labeled.drop(['Conto'], axis=1)
targets = data_labeled.Conto
#print(features.shape)


X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=8), n_estimators=100)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_pred, y_test))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#accuracy intorno a 0.94/0.95, come RF, però non funziona la feature selection. Se uso le 36 features selezionate da RF l'accuracy cala di circa 1%,
#probabilmente perchè non sono le features adatte ad AdaBoost.
#L'accuracy sale fino a 0.96/0.97 se la dimensione del training viene aumentata (90% dei dati) a scapito di quella del test set, cosa che 
#non accade nel RF

0.9424860853432282
[0.9539749  0.93922652 0.88888889 0.93436293 1.         1.
 0.72727273 1.         0.97959184 1.         1.        ]
0.9424860853432282
0.9475743456357955
0.9422508830920644


In [ ]:
#quinto classificatore provato: SVC

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

features = data_labeled.drop(['Conto'], axis=1)

"""
scaler = StandardScaler()
features.PrezzoUnitario = scaler.fit_transform(np.array(features.PrezzoUnitario).reshape(-1, 1))
features.PrezzoTotale = scaler.fit_transform(np.array(features.PrezzoTotale).reshape(-1, 1))
features.AliquotaIVA = scaler.fit_transform(np.array(features.AliquotaIVA).reshape(-1, 1))
features.Year = scaler.fit_transform(np.array(features.Year).reshape(-1, 1))
features.Month = scaler.fit_transform(np.array(features.Month).reshape(-1, 1))
features.Day = scaler.fit_transform(np.array(features.Day).reshape(-1, 1))
"""

print(features.columns)
targets = data_labeled.Conto
#print(features.shape)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
clf = make_pipeline(StandardScaler(), SVC(gamma='scale'))
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_pred, y_test))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#performance intorno a 0.87, anche al variare di alcuni parametri come il coefficiente di regolarizzazione C e gamma.
#Non cambia praticamente niente se si normalizzano tutte le feature o solo quelle numeriche.
#Nei 2 modelli migliori (AdaBoost e RF) l'accuracy e F score sono allineati, nei modelli peggiori come appunto SVC, F score
#in modalità macro risente del fatto che alcune classi non vengono apprese e perciò hanno F score individuale uguale a 0.

Index(['PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA',
       'Similarity_Materie_Prime', 'Similarity_Materie_Consumo',
       'Similarity_Merci', 'Similarity_Merci_Prodotti', 'Year', 'Month', 'Day',
       ...
       'NumeroDDT_PKL.21.1321', 'NumeroDDT_PKL.21.1457',
       'NumeroDDT_PKL.21.161', 'NumeroDDT_PKL.21.195', 'NumeroDDT_PKL.21.471',
       'NumeroDDT_PKL.21.520', 'NumeroDDT_PKL.21.55', 'NumeroDDT_PKL.21.597',
       'NumeroDDT_PKL.21.914', 'NumeroDDT_PKL.21.97'],
      dtype='object', length=204)
0.8534322820037106
[0.87826087 0.88297872 0.9044586  0.81506849 0.         0.
 0.         1.         0.82758621 0.99130435 1.        ]
0.8534322820037106
0.6636052035971737
0.8332993756936149


In [ ]:
#controllo di quali classi sono presenti nel train set, nel test set e quali vengono effettivamente predette

import numpy as np

print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_pred))

['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/507'
 '66/25/509' '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490'
 '70/05/101']
['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/509'
 '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490']
['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/509'
 '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490']


In [ ]:
#controllo della numerosità delle varie classi nei set
res = {}
unique, counts = np.unique(y_train, return_counts=True)
for i,j in zip(unique, counts):
  res[i] = {'train' : j}

unique, counts = np.unique(y_test, return_counts=True)
for i,j in zip(unique, counts):
  res[i]['test'] = j

unique, counts = np.unique(y_pred, return_counts=True)
for i,j in zip(unique, counts):
  res[i]['pred'] = j

for k,v in res.items():
  print(k,v)


66/05/006 {'train': 256, 'test': 121, 'pred': 121}
66/20/005 {'train': 166, 'test': 89, 'pred': 90}
66/25/005 {'train': 200, 'test': 86, 'pred': 74}
66/25/006 {'train': 285, 'test': 129, 'pred': 142}
66/25/506 {'train': 14, 'test': 20, 'pred': 20}
66/25/507 {'train': 13}
66/25/509 {'train': 6, 'test': 2, 'pred': 2}
66/30/015 {'train': 17, 'test': 6, 'pred': 4}
66/30/017 {'train': 5, 'test': 3, 'pred': 3}
66/30/055 {'train': 39, 'test': 24, 'pred': 24}
68/05/025 {'train': 236, 'test': 57, 'pred': 57}
68/05/490 {'train': 6, 'test': 2, 'pred': 2}
70/05/101 {'train': 12}


In [ ]:
#confusion matrix per analisi degli errori piu comuni

from sklearn.metrics import confusion_matrix
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

#con l'aggiunta delle informazioni estratte dalla descrizione le performance migliorano ma gli errori restano relativi alle stesse
#categorie di prima, in misura minore. Fortunatamente queste features aggiuntive non creano ocnfusione nelle altre classi

{'66/05/006': 121, '66/20/005': 89, '66/25/005': 86, '66/25/006': 129, '66/25/506': 20, '66/25/509': 2, '66/30/015': 6, '66/30/017': 3, '66/30/055': 24, '68/05/025': 57, '68/05/490': 2}


array([[115,   4,   0,   2,   0,   0,   0,   0,   0,   0,   0],
       [  3,  86,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,  71,  15,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   3, 124,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  20,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   2,   0,   0,   0,   0,   0],
       [  0,   2,   0,   0,   0,   0,   4,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  24,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  57,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2]])

In [ ]:
#analisi dell'effettiva informazione portata dalle features di similarity

print(data_labeled.groupby('Conto').mean()[['Similarity_Materie_Prime', 'Similarity_Materie_Consumo']])
print(data_labeled.groupby('Conto').mean()[['Similarity_Merci', 'Similarity_Merci_Prodotti']])

#per 3 classi su 4 effettivamente le features di similarità si comportano come previsto. Andrebbe migliorata la scelta delle
#parole tipiche delle descrizioni del gruppo 66/25/005, che in effetti è il gruppo su cui vengono ancora commessi la maggior parte
#degli errori, scambiandola con la classe 66/25/006

           Similarity_Materie_Prime  Similarity_Materie_Consumo
Conto                                                          
66/05/006                  0.418216                    0.252536
66/20/005                  0.266449                    0.326461
66/25/005                  0.256683                    0.221359
66/25/006                  0.298135                    0.245962
66/25/506                  0.255085                    0.217104
66/25/507                  0.388502                    0.265801
66/25/509                  0.437905                    0.340030
66/30/015                  0.237376                    0.289640
66/30/017                  0.227693                    0.252930
66/30/055                  0.200202                    0.245510
68/05/025                  0.193863                    0.193651
68/05/490                  0.209511                    0.264706
70/05/101                  0.201765                    0.201089
           Similarity_Merci  Similarity_

In [ ]:
#per verificare se le performance migliorano in qualunque delle configurazioni successive, voglio avere gli stess1 identici
#test set e training set. 

data_test = data_labeled.loc[1255:1793,:]
print(data_test)

data_train = data_labeled.loc[:1254,:]
print(data_train)

     Numero  PrezzoUnitario  ...  NumeroDDT_PKL.21.97      Conto
1255  16119          0.0000  ...                    0  66/25/006
1256  16119          1.3500  ...                    0  66/25/006
1257  16119          0.0000  ...                    0  66/25/006
1258  16119          4.3500  ...                    0  66/25/006
1259  16119          0.0000  ...                    0  66/25/006
...     ...             ...  ...                  ...        ...
1789    863          0.9836  ...                    0  66/20/005
1790    863          1.1475  ...                    0  66/20/005
1791    863          0.5328  ...                    0  66/20/005
1792   1682          0.0000  ...                    0  66/05/006
1793   1682          0.0880  ...                    0  66/05/006

[539 rows x 207 columns]
              Numero  PrezzoUnitario  ...  NumeroDDT_PKL.21.97      Conto
0     CI202074250324          17.980  ...                    0  66/25/005
1     CI202074250324           0.053  ...     

In [ ]:
#ora creo il dataset per il semi supervised training con le 1255 unità non usate per il test e tutte quelle con 'Conto' NaN.
#Inoltre viene performato l'encoding delle labels per non avere stringhe e -1 ma solo interi

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data_labeled['Conto'])
data_labeled['Conto'] = le.transform(data_labeled['Conto'])
data_semi = pd.concat(objs=[data_train, data_unlabeled])
data_semi.reset_index(inplace=True)
data_semi = data_semi.drop('index', axis=1)
data_semi = data_semi.fillna(-1)
print(data_semi)


      PrezzoUnitario  PrezzoTotale  ...  NumeroDDT_PKL.21.97  Conto
0             17.980        308.16  ...                    0      2
1              0.053          1.06  ...                    0      2
2             19.000        369.36  ...                    0      3
3              0.000          0.00  ...                    0      3
4              1.350         27.00  ...                    0      3
...              ...           ...  ...                  ...    ...
5311           0.000          0.00  ...                    0     -1
5312           0.000          0.00  ...                    0     -1
5313           0.000          0.00  ...                    0     -1
5314           0.000          0.00  ...                    0     -1
5315          10.000         10.00  ...                    0     -1

[5316 rows x 205 columns]


In [ ]:
#ora il dataset data_semi è pronto per il semi supervised learning dato che ha dei -1 al posto dei target mancanti

from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn import preprocessing

#clf = make_pipeline(StandardScaler(), SVC(gamma='scale', probability=True))
#clf = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=5)
clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=8), n_estimators=100)
calibrated_clf = CalibratedClassifierCV(base_estimator=clf)

X = data_semi.drop('Conto', axis=1)
y = data_semi.Conto
calibrated_clf.fit(X, y)

self_training_model = SelfTrainingClassifier(calibrated_clf, threshold=0.95, max_iter=2)
self_training_model.fit(X, y)

SelfTrainingClassifier(base_estimator=CalibratedClassifierCV(base_estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),
                                                                                               n_estimators=100)),
                       max_iter=2, threshold=0.95)

In [ ]:
#controllo di quali samples sono stati labeled e a che iterazione

for i in range(len(self_training_model.labeled_iter_)):
  print(self_training_model.labeled_iter_[i])
  print(self_training_model.transduction_[i])


In [ ]:
#creazione del nuovo training set con l'aggiunta dei nuovi dati pseudo supervisionati

mask = self_training_model.labeled_iter_ != -1
data_train = data_semi.loc[mask,]
data_train.Conto = self_training_model.transduction_[mask]
print(data_train)

      PrezzoUnitario  PrezzoTotale  ...  NumeroDDT_PKL.21.97  Conto
0             17.980        308.16  ...                    0      2
1              0.053          1.06  ...                    0      2
2             19.000        369.36  ...                    0      3
3              0.000          0.00  ...                    0      3
4              1.350         27.00  ...                    0      3
...              ...           ...  ...                  ...    ...
5162           0.300          0.30  ...                    0      9
5205           0.300          0.30  ...                    0      9
5239           0.300          0.30  ...                    0      9
5240           0.000          0.00  ...                    0      0
5310           0.300          0.30  ...                    0      9

[1860 rows x 205 columns]


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#controllo delle performance predittive grazie all'aumento dei samples supervisionati, usando lo stesso test set
#del caso senza semi supervised learning

from sklearn.metrics import accuracy_score

print(accuracy_score(self_training_model.predict(data_test.drop('Conto', axis=1)), le.transform(data_test.Conto)))

#purtroppo le performance non migliorano come sperato a prescindere da quanto venga ampliato il training set grazie al semi supervised learning,
#anche se vengono recuperate solo labels ad alta confidenza

0.9461966604823747
